# Tutorial PyBind11
## Useful links
* https://pybind11.readthedocs.io/en/stable/index.html  (the official pybind11 documentation)
* https://github.com/scikit-build/scikit-build-sample-projects (pybind11 example module using Python scikit-build as a buildsystem)
* https://github.com/tdegeus/pybind11_examples (examples on passing data between Python and C++ using pybind11)

## Using scikit-build
### build commands
```bash
python setup.py develop --user test # compiles in release mode, installs in develop mode and executes all tests
python setup.py develop --user --build-type=Debug # compiles in debug mode
```
### debugging in Visual Studio
1. Create a separate conda environment used for debugging
1. In order to debug the extension you have to build numpy from scratch. Beforehand you have to uninstall numpy, so run **conda uninstall numpy**.
1. Download the latest numpy release from https://github.com/numpy/numpy/releases, unpack the ZIP, change into the directory and call **pip install . --user**
1. Change into your python extension folder and call **python setup.py develop --user --build-type=Debug -G "Visual Studio 15 2017 Win64"** to build a Visual Studio Solution
1. Open the solution located in **_skbuild/win-amd64-3.7/cmake-build/\<xyz\>.sln**
1. Add a PythonApplication project to the solution and choose the right Conda environment
1. Open the project settings and enable **native debugging** in the tab Debug and check again the correct Interpreter in tab General
1. Set the PythonApplication as your StartUp Project
1. Write your Python code and set breakpoints in your Python and/or C++ code as required
1. Start Debugging

## Development hints

### Mixing C++/Python in one module
Take care on writing the \_\_init\_\_.py files
* create subfolders according to the submodule name, e.g. submodule
* in the PyBind11 code name prefix the submodule name with an \_, e.g. \_sbmodule
* put a \_\_init\_\_.py into the submodule folder and write something like this

```python
from _pyext._submodule import *
from .core import update   # imports update function defined in submodule/core.py 
```

* your main \_\_init\_\_.py should look like this

```python
import os, sys
sys.path.append(os.path.dirname(os.path.abspath( __file__ )))
from _pyext import _submodule as submodule
from . import submodule
```

### Splitting module definitions in multiple files
Simply put in you main.cpp
```c++
PYBIND11_MODULE(_pyext, m) {
  pyext::a::init_module(m);
}
```
Then you have to implement in another C++ file
```c++
void pyext::a::init_module(py::module& m)
{
  py::module m_a = m.def_submodule("_a");
  // --- functions
  m_a.def("prepare", &pyext::a::prepare);
}
```

### PyBind11 doesn't support natively C-style arrays
Suppose you want to have read/write access to the following C data structure.
```c++
typedef struct {
    float number;
    int   array[10];
} StructWithCStyleArrays_t;
```

Implement the following template functions 
```c++
/// function to read from a C-style array of size N into a std::array which is compatible to pybind11
template<typename T, int N>
std::array<T, N> getCArray(const T* arr)
{
  std::array<T, N> ndarray;
  std::memcpy(ndarray.data(), arr, N * sizeof(arr[0]));
  return ndarray;
}

/// function to write to a C-style array of size N given a std::array which is compatible to pybind11
template<typename T, int N>
void setCArray(T* arr, const std::array<T, N>& ndarray)
{
  std::memcpy(arr, ndarray.data(), N * sizeof(T));
}
```
and use them in your binding code as following
```c++
PYBIND11_MODULE(_pytp, m) {
  py::class_<StructWithCStyleArrays_t>(m, "StructWithCStyleArrays_t")
    .def(py::init<>())
    .def_readwrite("number", &StructWithCStyleArrays_t::number)
    .def_property("array",
      [](const StructWithCStyleArrays_t& p)->std::array<int, 10>     { return getCArray<int, 10>(p.array); },
      [](StructWithCStyleArrays_t& p, const std::array<int, 10> val) { setCArray<int, 10>(p.array, val); })
    }
}
```

### Read/Write access to nested structures
Suppose StructB_t is a nested attribute of StructA_t with name **B**.
```c++
    .def_property("nested_structure",
      [](const StructA_t& p)->StructB_t                            { return p.B; },
      [](StructA_t& p, const StructB_t& val)                       { p.B = val; });
```

### Initialize all attributes of a C/C++ structure given a dictionary with keys according to the attribute names
```python
def get_struct_members(struct: object) -> list:
    """
    Returns a list of all member variables of the given struct or class object
    @param struct: a class or struct object
    @return: list of member names
    """
    members = [x for x in dir(struct) if x[:2] != '__']
    return members


def init_struct_with_dict(struct: object, dictionary: dict, struct_key: str) -> object:
    """
    Initializes recursively struct members of a Python class (pure or pybind11 class) with public read/write
    access with data from a dictionary. The dictionary contains all the struct members as keys prefixed with a
    root struct key name.
    @param struct: class or struct object to be filled with data
    @param dictionary: input dictionary with columns according to the struct member names
    @param struct_key: prefix string for column names available in the dictionary
    @return: initialized struct object with data from dictionary
    """
    if struct_key and not struct_key.endswith('.'):
        struct_key = struct_key + '.'

    # contains atomic members like float, int attributes as well as struct members
    struct_members = set([struct_key + x for x in get_struct_members(struct)])
    # the dictionary keys represent the leaves of all structures, i.e. atomic attributes
    # thus the intersection with struct_members will give us all atomic members which can be directly set
    atomic_members = struct_members.intersection(dictionary.keys())
    # the remaining members are struct members requiring a recursive init call
    complex_members = struct_members - atomic_members

    # init all atomic members directly
    for dict_key in atomic_members:
        name = dict_key.split('.')[-1]
        call_str = f'struct.{name} = {dictionary[dict_key]}'
        exec(call_str)

    # init recursively all substructures
    for dict_key in complex_members:
        name = dict_key.split('.')[-1]
        call_str = f"struct.{name} = init_struct_with_dict( struct.{name}, dictionary, '{dict_key}' )"
        exec(call_str)

    return struct
```
With those functions you can easily write
```python
data = pyext.StructA_t()  # create an instance of a C/C++ structure
data = pyext.init_struct_with_dict(data, data_dict, '')
```
which then looks for matching keys in the dictionary to do the assignment to the attribute. Further this function will initialize all nested structure recursively.